# 基于特征的经典方法效果

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.metrics import accuracy_score
import matplotlib
matplotlib.use('Agg')  # 使用非交互式后端
import matplotlib.pyplot as plt

# ==========================================
# 工具函数：计算模型参数量
# ==========================================
def count_parameters(model):
    """计算 PyTorch 模型的可训练参数量"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# ==========================================
# 1. 数据加载与预处理
# ==========================================
print("--- 1. Loading and Preprocessing Data ---")
file_path = r'./data/data_koopman_6.txt' 
# file_path = r'./data/dataF.txt' 

try:
    data = np.loadtxt(file_path, delimiter=',')
except OSError:
    print(f"Error: File {file_path} not found. Generating dummy data for demonstration.")
    # 生成模拟数据: 1000个样本, 6个特征, 1个标签(0或1)
    data = np.random.rand(1000, 7)
    data[:, -1] = np.random.randint(0, 2, 1000)

# 分离特征 (6维) 和标签
X = data[:, :-1]
y = data[:, -1]

# 标签映射
unique_labels = np.unique(y)
label_map = {unique_labels[0]: 0, unique_labels[1]: 1}
y_mapped = np.vectorize(label_map.get)(y)

# 数据拆分 (70% 训练, 30% 测试)
X_train, X_test, y_train, y_test = train_test_split(X, y_mapped, test_size=0.3, random_state=42)

# 标准化 (归一化到 0-pi)
scaler = MinMaxScaler(feature_range=(0, np.pi))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 转换为 Tensor
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

print(f"Data Shape: {X.shape}")

# ==========================================
# 2. 定义模型结构
# ==========================================

# --- 模型 A: 深度全连接网络 (MLP/FCNN) ---
class DeepKoopmanNet(nn.Module):
    def __init__(self, num_features):
        super(DeepKoopmanNet, self).__init__()
        # 6层全连接结构
        self.layers = nn.ModuleList([
            nn.Linear(num_features, 6),
            nn.Linear(6, 6),
            nn.Linear(6, 6),
            # nn.Linear(6, 6),
            # nn.Linear(6, 6),
            # nn.Linear(6, 6)
        ])
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
            x = torch.relu(x) # 可选：添加激活函数增加非线性能力
        
        # 输出聚合
        out0 = x[:, ::2].mean(dim=1)
        out1 = x[:, 1::2].mean(dim=1)
        out = torch.stack([out0, out1], dim=1)
        return out

# --- 模型 B: 1D 卷积神经网络 (1D-CNN) [新增] ---
class Koopman1DCNN(nn.Module):
    def __init__(self, num_features=6):
        super(Koopman1DCNN, self).__init__()
        
        # 输入: (Batch, 1, 6) -> 视作长度为6的序列
        
        self.features = nn.Sequential(
            # Conv1: 1 -> 4 channels, kernel=3
            nn.Conv1d(in_channels=1, out_channels=4, kernel_size=3, padding=1),
            nn.BatchNorm1d(4),
            nn.ReLU(),
            
            # Conv2: 4 -> 8 channels, kernel=3
            nn.Conv1d(4, 8, kernel_size=3, padding=1),
            nn.BatchNorm1d(8),
            nn.ReLU(),
            
            # Conv3: 8 -> 16 channels, kernel=3
            nn.Conv1d(8, 16, kernel_size=3, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            
            # Global Average Pooling: (Batch, 16, 6) -> (Batch, 16, 1)
            nn.AdaptiveAvgPool1d(1)
        )
        
        self.classifier = nn.Linear(16, 2)

    def forward(self, x):
        # x shape: (Batch, 6) -> unsqueeze -> (Batch, 1, 6)
        x = x.unsqueeze(1)
        x = self.features(x)
        x = x.view(x.size(0), -1) # Flatten -> (Batch, 16)
        x = self.classifier(x)
        return x

# ==========================================
# 3. 模型初始化与参数统计
# ==========================================
print("\n--- 2. Model Initialization & Parameter Count ---")

# 1. MLP
model_mlp = DeepKoopmanNet(num_features=6)
mlp_params = count_parameters(model_mlp)
print(f"[MLP] Deep Fully Connected (Depth=6): {mlp_params} params")

# 2. CNN
model_cnn = Koopman1DCNN(num_features=6)
cnn_params = count_parameters(model_cnn)
print(f"[CNN] 1D-CNN (3 Conv Layers): {cnn_params} params")

# 3. SVM
svm_clf = svm.SVC(kernel='linear', C=8)
svm_params_est = X_train.shape[1] + 1
print(f"[SVM] Linear SVM: ~{svm_params_est} params")

# ==========================================
# 4. 通用训练函数
# ==========================================
def train_pytorch_model(model, model_name, X_train, y_train, X_test, y_test, epochs=500, batch_size=1083, lr=0.0005):
    print(f"\n--- Training {model_name} ---")
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    losses = []
    train_accs = []
    test_accs = []
    
    model.train()
    num_samples = X_train.shape[0]
    num_batches = max(1, num_samples // batch_size)

    for epoch in range(epochs):
        for batch_idx in range(num_batches):
            start_idx = batch_idx * batch_size
            end_idx = min(start_idx + batch_size, num_samples)
            
            # 数据切片
            X_batch = X_train[start_idx:end_idx]
            y_batch = y_train[start_idx:end_idx]

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            # 记录 (每个 Epoch 记录一次)
            if batch_idx == 0:
                # Train Acc
                _, preds = torch.max(outputs, 1)
                acc = accuracy_score(y_batch.numpy(), preds.numpy())
                
                # Test Acc
                model.eval()
                with torch.no_grad():
                    outputs_test = model(X_test)
                    _, preds_test = torch.max(outputs_test, 1)
                    test_acc = accuracy_score(y_test.numpy(), preds_test.numpy())
                model.train()

                losses.append(loss.item())
                train_accs.append(acc)
                test_accs.append(test_acc)
                
                if (epoch + 1) % 100 == 0:
                    print(f"Epoch {epoch+1}: Loss={loss.item():.4f}, Train={acc:.3f}, Test={test_acc:.3f}")
    
    return losses, train_accs, test_accs


# ==========================================
# 5. 执行训练 & 保存数据
# ==========================================

# --- 1. 训练 MLP ---
losses_mlp, train_acc_mlp, test_acc_mlp = train_pytorch_model(
    model_mlp, "MLP", X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor,
    epochs=500, batch_size=1083, lr=0.001
)

# 【新增】保存 MLP 日志
log_mlp = {
    'name': 'MLP (Koopman 6D)',
    'params': mlp_params,
    'loss': losses_mlp,
    'train_acc': train_acc_mlp,
    'test_acc': test_acc_mlp,
    'epochs': list(range(1, len(losses_mlp) + 1))
}
np.save('log_mlp_koopman_6d.npy', log_mlp)
print(">>> MLP 训练日志已保存至: log_mlp_koopman_6d.npy")


# --- 2. 训练 CNN ---
losses_cnn, train_acc_cnn, test_acc_cnn = train_pytorch_model(
    model_cnn, "CNN", X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor,
    epochs=500, batch_size=1083, lr=0.001
)

# 【新增】保存 CNN 日志
log_cnn = {
    'name': 'CNN (Koopman 6D)',
    'params': cnn_params,
    'loss': losses_cnn,
    'train_acc': train_acc_cnn,
    'test_acc': test_acc_cnn,
    'epochs': list(range(1, len(losses_cnn) + 1))
}
np.save('log_cnn_koopman_6d.npy', log_cnn)
print(">>> CNN 训练日志已保存至: log_cnn_koopman_6d.npy")

# --- 3. 训练 SVM (作为参考，不画曲线) ---
print("\n--- Training SVM ---")
svm_clf.fit(X_train_scaled, y_train)
svm_test_acc = svm_clf.score(X_test_scaled, y_test)
print(f"SVM Test Acc: {svm_test_acc:.3f}")
# # ==========================================
# # 5. 执行训练
# # ==========================================

# # 训练 MLP
# losses_mlp, train_acc_mlp, test_acc_mlp = train_pytorch_model(
#     model_mlp, "MLP", X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor
# )

# # 训练 CNN
# losses_cnn, train_acc_cnn, test_acc_cnn = train_pytorch_model(
#     model_cnn, "CNN", X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor
# )

# # 训练 SVM
# print("\n--- Training SVM ---")
# svm_clf.fit(X_train_scaled, y_train)
# svm_train_acc = svm_clf.score(X_train_scaled, y_train)
# svm_test_acc = svm_clf.score(X_test_scaled, y_test)
# print(f"SVM Test Acc: {svm_test_acc:.3f}")

# # ==========================================
# # 6. 绘图对比
# # ==========================================
# print("\n--- Plotting Results ---")
# fig, ax1 = plt.subplots(figsize=(10, 6))

# epochs_range = range(1, len(losses_mlp) + 1)

# # 左轴：Loss
# ax1.set_xlabel('Epochs')
# ax1.set_ylabel('Loss', color='black')
# ax1.plot(epochs_range, losses_mlp, color='tab:red', linestyle=':', label="MLP Loss", alpha=0.5)
# ax1.plot(epochs_range, losses_cnn, color='tab:green', linestyle=':', label="CNN Loss", alpha=0.5)
# ax1.tick_params(axis='y', labelcolor='black')

# # 右轴：Accuracy
# ax2 = ax1.twinx()
# ax2.set_ylabel('Test Accuracy', color='tab:blue')

# # 绘制各模型 Accuracy
# # 1. MLP
# ax2.plot(epochs_range, test_acc_mlp, color='tab:red', linestyle='-', linewidth=1.5, 
#          label=f"MLP (Params={mlp_params}): {test_acc_mlp[-1]:.3f}")
# # 2. CNN
# ax2.plot(epochs_range, test_acc_cnn, color='tab:green', linestyle='-', linewidth=1.5, 
#          label=f"CNN (Params={cnn_params}): {test_acc_cnn[-1]:.3f}")
# # 3. SVM 基线
# ax2.axhline(y=svm_test_acc, color='tab:purple', linestyle='--', linewidth=2, 
#             label=f"SVM (Params={svm_params_est}): {svm_test_acc:.3f}")

# ax2.tick_params(axis='y', labelcolor='tab:blue')

# # 图例与标题
# plt.title("Koopman Feature Classification: MLP vs CNN vs SVM")
# lines1, labels1 = ax1.get_legend_handles_labels()
# lines2, labels2 = ax2.get_legend_handles_labels()
# ax2.legend(lines1 + lines2, labels1 + labels2, loc="lower right")

# plt.grid(True, alpha=0.3)
# plt.savefig("model_comparison_with_cnn.pdf")
# print("Plot saved to model_comparison_with_cnn.pdf")

--- 1. Loading and Preprocessing Data ---
Data Shape: (4763, 6)

--- 2. Model Initialization & Parameter Count ---
[MLP] Deep Fully Connected (Depth=6): 126 params
[CNN] 1D-CNN (3 Conv Layers): 610 params
[SVM] Linear SVM: ~7 params

--- Training MLP ---
Epoch 100: Loss=0.2013, Train=0.944, Test=0.935
Epoch 200: Loss=0.1166, Train=0.970, Test=0.965
Epoch 300: Loss=0.1059, Train=0.973, Test=0.966
Epoch 400: Loss=0.1029, Train=0.976, Test=0.968
Epoch 500: Loss=0.1015, Train=0.976, Test=0.969
>>> MLP 训练日志已保存至: log_mlp_koopman_6d.npy

--- Training CNN ---
Epoch 100: Loss=0.0994, Train=0.976, Test=0.969
Epoch 200: Loss=0.0850, Train=0.978, Test=0.968
Epoch 300: Loss=0.0798, Train=0.980, Test=0.969
Epoch 400: Loss=0.0780, Train=0.980, Test=0.969
Epoch 500: Loss=0.0773, Train=0.980, Test=0.969
>>> CNN 训练日志已保存至: log_cnn_koopman_6d.npy

--- Training SVM ---
SVM Test Acc: 0.966


In [17]:


import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, accuracy_score
import matplotlib
matplotlib.use('Agg') # 服务器端绘图
import matplotlib.pyplot as plt
import os

# ================= 配置 =================
CONFIG = {
    'data_path': r'./data/data_koopman_6.txt',
    'batch_size': 32,
    'epochs': 300, # Koopman数据收敛快，100轮足够
    'lr': 0.001,
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'seed': 42,
    'save_fig_path': 'CNN_on_Koopman_Visualization.pdf'
}

# ==============================================================================
# 1. 模型定义 (Modified CNN)
# ==============================================================================

class Koopman1DCNN(nn.Module):
    def __init__(self, num_features=6):
        super(Koopman1DCNN, self).__init__()
        
        # 输入: (Batch, 1, 6)
        
        # Layer 1
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=4, kernel_size=3, padding=1),
            nn.BatchNorm1d(4),
            nn.ReLU()
        ) # Out: (Batch, 4, 6)
        
        # Layer 2 (中间特征点)
        self.conv2 = nn.Sequential(
            nn.Conv1d(4, 8, kernel_size=3, padding=1),
            nn.BatchNorm1d(8),
            nn.ReLU()
        ) # Out: (Batch, 8, 6)
        
        # Layer 3 (最终特征点)
        self.conv3 = nn.Sequential(
            nn.Conv1d(8, 16, kernel_size=3, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1) # Global Avg Pooling
        ) # Out: (Batch, 16, 1)
        
        self.classifier = nn.Linear(16, 2)

    def forward(self, x):
        # x shape: (Batch, 6) -> (Batch, 1, 6)
        x = x.unsqueeze(1)
        
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        x = x.view(x.size(0), -1) # Flatten -> (Batch, 16)
        x = self.classifier(x)
        return x

    # 【新增】特征提取方法
    def extract_features(self, x):
        """提取中间层和最终潜层特征"""
        x = x.unsqueeze(1)
        
        # Pass Layer 1
        out1 = self.conv1(x)
        
        # Pass Layer 2 -> Extract Intermediate
        out2 = self.conv2(out1)
        # 将 (Batch, 8, 6) 展平为 (Batch, 48) 用于可视化
        feat_inter = out2.view(out2.size(0), -1)
        
        # Pass Layer 3 -> Extract Final Latent
        out3 = self.conv3(out2) # (Batch, 16, 1)
        feat_final = out3.view(out3.size(0), -1) # (Batch, 16)
        
        return feat_inter, feat_final

# ==============================================================================
# 2. 训练与数据加载流程
# ==============================================================================

def train_and_get_features():
    print(">>> Loading Data...")
    try:
        data = np.loadtxt(CONFIG['data_path'], delimiter=',')
    except:
        print("Warning: Data file not found, generating dummy data.")
        data = np.random.rand(500, 7)
        data[:, -1] = np.random.randint(0, 2, 500)

    X = data[:, :-1]
    y = data[:, -1]

    # 标签映射
    unique_labels = np.unique(y)
    label_map = {unique_labels[0]: 0, unique_labels[1]: 1}
    y_mapped = np.vectorize(label_map.get)(y)

    # Split
    X_train, X_test, y_train, y_test = train_test_split(X, y_mapped, test_size=0.3, random_state=CONFIG['seed'])

    # 归一化 (0-pi)
    scaler = MinMaxScaler(feature_range=(0, np.pi))
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 转 Tensor
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(CONFIG['device'])
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(CONFIG['device'])
    y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(CONFIG['device'])
    y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(CONFIG['device'])

    # 初始化模型
    model = Koopman1DCNN(num_features=6).to(CONFIG['device'])
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'])
    criterion = nn.CrossEntropyLoss()

    print(f">>> Training CNN on {CONFIG['device']}...")
    model.train()
    
    # 简化的训练循环
    for epoch in range(CONFIG['epochs']):
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 20 == 0:
            acc = (outputs.argmax(1) == y_train_tensor).float().mean()
            print(f"Epoch {epoch+1}/{CONFIG['epochs']} | Loss: {loss.item():.4f} | Train Acc: {acc:.3f}")

    # --- 提取可视化特征 (仅使用测试集) ---
    print(">>> Extracting Features for Visualization...")
    model.eval()
    with torch.no_grad():
        # (a) Input Space (就是归一化后的 Koopman 特征)
        feat_input = X_test_scaled
        
        # (b) & (c) Model Features
        feat_inter, feat_final = model.extract_features(X_test_tensor)
        feat_inter = feat_inter.cpu().numpy()
        feat_final = feat_final.cpu().numpy()
        
    return feat_input, feat_inter, feat_final, y_test

# ==============================================================================
# 3. 可视化流程 (Visualization Pipeline)
# ==============================================================================

def plot_feature_evolution(feat_input, feat_inter, feat_final, labels):
    print(">>> Running t-SNE and Plotting...")
    
    data_map = [
        ('Input Koopman Space\n(6-dim Features)', feat_input),
        ('CNN Intermediate Space\n(Layer 2 Output)', feat_inter),
        ('CNN Latent Space\n(Global Avg Pooling)', feat_final)
    ]
    
    # 样式配置 (保持与量子论文一致)
    plt.style.use('seaborn-v0_8-paper')
    plt.rcParams.update({
        "font.family": "serif",
        "font.serif": ["Times New Roman"],
        "font.size": 12,
        "axes.labelsize": 14,
        "legend.fontsize": 12,
        "figure.titlesize": 16
    })
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    colors = ['#1f77b4', '#ff7f0e'] # Blue, Orange
    class_names = ['Normal', 'Disruption']
    
    for i, (title, data) in enumerate(data_map):
        ax = axes[i]
        
        # t-SNE (针对小数据集的参数优化)
        # perplexity: 数据少时设小(30)，多时设大(50)
        perp = min(30, len(data)-1)
        tsne = TSNE(
            n_components=2, 
            perplexity=50,          # 建议尝试 50 或 80，消除长条纹，让簇更圆润
            early_exaggeration=20,  # 增大此值，强行拉大类间距离，视觉更震撼
            learning_rate='auto',   # 自动学习率
            init='pca',             # 使用 PCA 初始化，保留全局结构，图更整齐
            max_iter=1000,            # 增加迭代次数，确保收敛
            random_state=42
        )
        emb = tsne.fit_transform(data)
        
        # S-Score
        try:
            score = silhouette_score(data, labels)
        except: score = 0
        
        # Scatter Plot
        for lbl_idx, color in enumerate(colors):
            mask = (labels == lbl_idx)
            # ax.scatter(emb[mask, 0], emb[mask, 1], c=color, label=class_names[lbl_idx],
            #            alpha=0.75, s=40, edgecolors='w', linewidth=0.3)
            ax.scatter(
                emb[mask, 0], emb[mask, 1], 
                c=color, 
                label=class_names[lbl_idx],
                alpha=0.75,   # 透明度从 0.6 提高到 0.75，让颜色更实，对比度更高
                s=30,         # 点的大小从 20 提高到 30，让点更清晰
                edgecolors='w', # 加白色描边
                linewidth=0.3   # 描边细一点
            )  
            
        ax.set_title(f"({chr(97+i)}) {title}", fontweight='bold')
        ax.set_xticks([])
        ax.set_yticks([])
        
        # 指标框
        ax.text(0.05, 0.92, f'S-Score: {score:.3f}', transform=ax.transAxes,
                bbox=dict(facecolor='white', alpha=0.9, edgecolor='gray', boxstyle='round'))

    # Global Legend
    handles, _ = axes[0].get_legend_handles_labels()
    fig.legend(handles, class_names, loc='lower center', ncol=2, bbox_to_anchor=(0.5, 0.0), frameon=True)
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.15)
    plt.savefig(CONFIG['save_fig_path'], dpi=300)
    print(f">>> Plot saved to {CONFIG['save_fig_path']}")

# ==============================================================================
# Main
# ==============================================================================
if __name__ == "__main__":
    # 1. 训练并提取特征
    f_in, f_mid, f_out, lbls = train_and_get_features()
    
    # 2. 画图
    plot_feature_evolution(f_in, f_mid, f_out, lbls)



>>> Loading Data...
>>> Training CNN on cpu...
Epoch 20/300 | Loss: 0.4231 | Train Acc: 0.901
Epoch 40/300 | Loss: 0.3266 | Train Acc: 0.901
Epoch 60/300 | Loss: 0.2496 | Train Acc: 0.933
Epoch 80/300 | Loss: 0.2045 | Train Acc: 0.953
Epoch 100/300 | Loss: 0.1750 | Train Acc: 0.960
Epoch 120/300 | Loss: 0.1548 | Train Acc: 0.963
Epoch 140/300 | Loss: 0.1406 | Train Acc: 0.967
Epoch 160/300 | Loss: 0.1309 | Train Acc: 0.969
Epoch 180/300 | Loss: 0.1240 | Train Acc: 0.970
Epoch 200/300 | Loss: 0.1188 | Train Acc: 0.970
Epoch 220/300 | Loss: 0.1150 | Train Acc: 0.971
Epoch 240/300 | Loss: 0.1121 | Train Acc: 0.972
Epoch 260/300 | Loss: 0.1098 | Train Acc: 0.972
Epoch 280/300 | Loss: 0.1079 | Train Acc: 0.972
Epoch 300/300 | Loss: 0.1063 | Train Acc: 0.972
>>> Extracting Features for Visualization...
>>> Running t-SNE and Plotting...
>>> Plot saved to CNN_on_Koopman_Visualization.pdf


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, accuracy_score
import matplotlib
matplotlib.use('Agg') # 服务器端绘图
import matplotlib.pyplot as plt
import os

# ================= 配置 =================
CONFIG = {
    'data_path': r'./data/data_koopman_6.txt',
    'batch_size': 32,
    'epochs': 100, 
    'lr': 0.001,
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'seed': 42,
    'save_fig_path': 'MLP_on_Koopman_Visualization.pdf' # 修改保存文件名
}

# ==============================================================================
# 1. 模型定义 (MLP Version)
# ==============================================================================

class KoopmanMLP(nn.Module):
    def __init__(self, num_features=6):
        super(KoopmanMLP, self).__init__()
        
        # 输入: (Batch, 6)
        # 注意：MLP 不需要像 CNN 那样增加 Channel 维度
        
        # Layer 1
        self.layer1 = nn.Sequential(
            nn.Linear(num_features, 16),
            nn.BatchNorm1d(16),
            nn.ReLU()
        )
        
        # Layer 2 (中间特征提取点)
        self.layer2 = nn.Sequential(
            nn.Linear(16, 32),
            nn.BatchNorm1d(32),
            nn.ReLU()
        )
        
        # Layer 3 (最终特征提取点)
        self.layer3 = nn.Sequential(
            nn.Linear(32, 64),
            nn.BatchNorm1d(64),
            nn.ReLU()
            # MLP 不需要 Global Avg Pooling，因为它本身就是全连接的
        )
        
        self.classifier = nn.Linear(64, 2)

    def forward(self, x):
        # x shape: (Batch, 6)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        x = self.classifier(x)
        return x

    # 【特征提取方法】保持接口与 CNN 一致
    def extract_features(self, x):
        """提取中间层和最终潜层特征"""
        # x shape: (Batch, 6)
        
        # Pass Layer 1
        out1 = self.layer1(x)
        
        # Pass Layer 2 -> Extract Intermediate
        out2 = self.layer2(out1)
        # (Batch, 32)
        feat_inter = out2 
        
        # Pass Layer 3 -> Extract Final Latent
        out3 = self.layer3(out2) 
        # (Batch, 64)
        feat_final = out3
        
        return feat_inter, feat_final

# ==============================================================================
# 2. 训练与数据加载流程
# ==============================================================================

def train_and_get_features():
    print(">>> Loading Data...")
    try:
        data = np.loadtxt(CONFIG['data_path'], delimiter=',')
    except:
        print("Warning: Data file not found, generating dummy data.")
        data = np.random.rand(500, 7)
        data[:, -1] = np.random.randint(0, 2, 500)

    X = data[:, :-1]
    y = data[:, -1]

    # 标签映射
    unique_labels = np.unique(y)
    label_map = {unique_labels[0]: 0, unique_labels[1]: 1}
    y_mapped = np.vectorize(label_map.get)(y)

    # Split
    X_train, X_test, y_train, y_test = train_test_split(X, y_mapped, test_size=0.3, random_state=CONFIG['seed'])

    # 归一化 (0-pi)
    scaler = MinMaxScaler(feature_range=(0, np.pi))
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 转 Tensor
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(CONFIG['device'])
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(CONFIG['device'])
    y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(CONFIG['device'])
    y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(CONFIG['device'])

    # 初始化模型 【改为 MLP】
    model = KoopmanMLP(num_features=6).to(CONFIG['device'])
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'])
    criterion = nn.CrossEntropyLoss()
    
    # 打印参数量
    total_params = sum(p.numel() for p in model.parameters())
    print(f">>> Model: KoopmanMLP | Parameters: {total_params}")

    print(f">>> Training MLP on {CONFIG['device']}...")
    model.train()
    
    # 简化的训练循环
    for epoch in range(CONFIG['epochs']):
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 20 == 0:
            acc = (outputs.argmax(1) == y_train_tensor).float().mean()
            print(f"Epoch {epoch+1}/{CONFIG['epochs']} | Loss: {loss.item():.4f} | Train Acc: {acc:.3f}")

    # --- 提取可视化特征 (仅使用测试集) ---
    print(">>> Extracting Features for Visualization...")
    model.eval()
    with torch.no_grad():
        # (a) Input Space (就是归一化后的 Koopman 特征)
        feat_input = X_test_scaled
        
        # (b) & (c) Model Features
        feat_inter, feat_final = model.extract_features(X_test_tensor)
        feat_inter = feat_inter.cpu().numpy()
        feat_final = feat_final.cpu().numpy()
        
    return feat_input, feat_inter, feat_final, y_test

# ==============================================================================
# 3. 可视化流程 (Visualization Pipeline)
# ==============================================================================

def plot_feature_evolution(feat_input, feat_inter, feat_final, labels):
    print(">>> Running t-SNE and Plotting...")
    
    # 修改标题为 MLP
    data_map = [
        ('Input Koopman Space\n(6-dim Features)', feat_input),
        ('MLP Intermediate Space\n(Layer 2 Output)', feat_inter),
        ('MLP Latent Space\n(Layer 3 Output)', feat_final)
    ]
    
    # 样式配置 (保持与量子论文一致)
    plt.style.use('seaborn-v0_8-paper')
    plt.rcParams.update({
        "font.family": "serif",
        "font.serif": ["Times New Roman"],
        "font.size": 12,
        "axes.labelsize": 14,
        "legend.fontsize": 12,
        "figure.titlesize": 16
    })
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    colors = ['#1f77b4', '#ff7f0e'] # Blue, Orange
    class_names = ['Normal', 'Disruption']
    
    for i, (title, data) in enumerate(data_map):
        ax = axes[i]
        
        # t-SNE (针对小数据集的参数优化)
        perp = min(30, len(data)-1)
        tsne = TSNE(
            n_components=2, 
            perplexity=50,          # 建议尝试 50 或 80，消除长条纹，让簇更圆润
            early_exaggeration=20,  # 增大此值，强行拉大类间距离，视觉更震撼
            learning_rate='auto',   # 自动学习率
            init='pca',             # 使用 PCA 初始化，保留全局结构，图更整齐
            max_iter=1000,            # 增加迭代次数，确保收敛
            random_state=42
        )
        emb = tsne.fit_transform(data)
        
        # S-Score
        try:
            score = silhouette_score(data, labels)
        except: score = 0
        
        # Scatter Plot
        for lbl_idx, color in enumerate(colors):
            mask = (labels == lbl_idx)
            ax.scatter(
                emb[mask, 0], emb[mask, 1], 
                c=color, 
                label=class_names[lbl_idx],
                alpha=0.75,   # 透明度从 0.6 提高到 0.75，让颜色更实，对比度更高
                s=30,         # 点的大小从 20 提高到 30，让点更清晰
                edgecolors='w', # 加白色描边
                linewidth=0.3   # 描边细一点
            )  
            
        ax.set_title(f"({chr(97+i)}) {title}", fontweight='bold')
        ax.set_xticks([])
        ax.set_yticks([])
        
        # 指标框
        ax.text(0.05, 0.92, f'S-Score: {score:.3f}', transform=ax.transAxes,
                bbox=dict(facecolor='white', alpha=0.9, edgecolor='gray', boxstyle='round'))

    # Global Legend
    handles, _ = axes[0].get_legend_handles_labels()
    fig.legend(handles, class_names, loc='lower center', ncol=2, bbox_to_anchor=(0.5, 0.0), frameon=True)
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.15)
    plt.savefig(CONFIG['save_fig_path'], dpi=300)
    print(f">>> Plot saved to {CONFIG['save_fig_path']}")

# ==============================================================================
# Main
# ==============================================================================
if __name__ == "__main__":
    # 1. 训练并提取特征
    f_in, f_mid, f_out, lbls = train_and_get_features()
    
    # 2. 画图
    plot_feature_evolution(f_in, f_mid, f_out, lbls)

>>> Loading Data...
>>> Model: KoopmanMLP | Parameters: 3122
>>> Training MLP on cpu...
Epoch 20/100 | Loss: 0.2007 | Train Acc: 0.963
Epoch 40/100 | Loss: 0.1372 | Train Acc: 0.967
Epoch 60/100 | Loss: 0.1147 | Train Acc: 0.971
Epoch 80/100 | Loss: 0.1039 | Train Acc: 0.973
Epoch 100/100 | Loss: 0.0981 | Train Acc: 0.974
>>> Extracting Features for Visualization...
>>> Running t-SNE and Plotting...
>>> Plot saved to MLP_on_Koopman_Visualization.pdf
